In [1]:
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.6 MB/s eta 0:00:0000:0100:01


In [2]:
import torch
from kaggle_secrets import UserSecretsClient
import transformers
import warnings

warnings.filterwarnings('ignore')

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
your_token = user_secrets.get_secret("HuggingFaceSecret")


tokenizer = transformers.AutoTokenizer.from_pretrained(model_id,use_auth_token=your_token)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=your_token,
    load_in_4bit=True,
    device_map="cuda",
)




tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [4]:
!pip install -q kaggle
# !kaggle datasets download -d prathamjyotsingh/planets-dataset
!kaggle datasets download -d rajugc/imdb-movies-dataset-based-on-genre
# !kaggle datasets download -d zed9941/top-500-indian-cities
# !kaggle datasets download -d lainguyn123/australia-car-market-data

Dataset URL: https://www.kaggle.com/datasets/rajugc/imdb-movies-dataset-based-on-genre
License(s): CC-BY-NC-SA-4.0
 99%|█████████████████████████████████████▋| 53.0M/53.4M [00:03<00:00, 16.7MB/s]
100%|██████████████████████████████████████| 53.4M/53.4M [00:03<00:00, 15.1MB/s]


In [5]:
!unzip imdb-movies-dataset-based-on-genre

Archive:  imdb-movies-dataset-based-on-genre.zip
  inflating: action.csv              
  inflating: adventure.csv           
  inflating: animation.csv           
  inflating: biography.csv           
  inflating: crime.csv               
  inflating: family.csv              
  inflating: fantasy.csv             
  inflating: film-noir.csv           
  inflating: history.csv             
  inflating: horror.csv              
  inflating: mystery.csv             
  inflating: romance.csv             
  inflating: scifi.csv               
  inflating: sports.csv              
  inflating: thriller.csv            
  inflating: war.csv                 


In [6]:
import pandas as pd
import numpy as np
df = pd.read_csv('action.csv')
df_cleaned = df[~(df['certificate'].isna() | df['rating'].isna() | np.isinf(df['rating']))]
df_cleaned=df.dropna()
df_cleaned=df_cleaned.iloc[:150]
print(df_cleaned.shape)

(150, 14)


In [7]:
input_text=[]
for x in df_cleaned['movie_name']:
  prompt=f"What is the movie {x} rated on imdb? And what was its age certification?"
  input_text.append(prompt)
tokenizer.pad_token=tokenizer.eos_token
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [8]:
with torch.no_grad():
    outputs = model(**inputs, output_hidden_states=True)
    hidden_states = outputs.hidden_states

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


In [9]:
num_layers=len(hidden_states)
first_layer_embeddings = hidden_states[0][:, -1, :]
last_layer_embeddings=hidden_states[-1][:,-1,:]
middle_layer_index=num_layers//2
middle_layer_embeddings=hidden_states[middle_layer_index][:, -1, :]
print("First layer embeddings shape:", first_layer_embeddings.shape)
print("Middle layer embeddings shape:", middle_layer_embeddings.shape)
print("Last layer embeddings shape:", last_layer_embeddings.shape)



First layer embeddings shape: torch.Size([150, 4096])
Middle layer embeddings shape: torch.Size([150, 4096])
Last layer embeddings shape: torch.Size([150, 4096])


In [10]:
X_first = first_layer_embeddings.numpy()
X_middle = middle_layer_embeddings.numpy()
X_last = last_layer_embeddings.numpy()

In [11]:
from sklearn.linear_model import LinearRegression, LogisticRegression,SGDRegressor,SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder

population=df_cleaned['rating']
continent=df_cleaned['certificate']
def train_and_evaluate(X, y_regression, y_classification):
    
    label_encoder = LabelEncoder()
    continent_encoded = label_encoder.fit_transform(continent)
    
    X_train, X_test, y_train_reg, y_test_reg, y_train_cls, y_test_cls = train_test_split(
        X, y_regression, y_classification, test_size=0.2, random_state=42
    )
    
# #     mse=0
    regressor = LinearRegression()
    regressor.fit(X_train, y_train_reg)
    y_pred_reg = regressor.predict(X_test)
    mse = mean_squared_error(y_test_reg, y_pred_reg)


    classifier= SGDClassifier(learning_rate='constant', eta0=0.01)
    classifier.fit(X_train, y_train_cls)
    y_pred_cls = classifier.predict(X_test)
    accuracy = accuracy_score(y_test_cls, y_pred_cls)

    return mse, accuracy


mse_first, acc_first = train_and_evaluate(X_first, population, continent)
mse_middle, acc_middle = train_and_evaluate(X_middle, population, continent)
mse_last, acc_last = train_and_evaluate(X_last, population, continent)

print("First Layer - MSE:", mse_first, "Accuracy:", acc_first)
print("Middle Layer - MSE:", mse_middle, "Accuracy:", acc_middle)
print("Last Layer - MSE:", mse_last, "Accuracy:", acc_last)

First Layer - MSE: 0.483177490234375 Accuracy: 0.5666666666666667
Middle Layer - MSE: 0.19373606363932294 Accuracy: 0.5333333333333333
Last Layer - MSE: 0.21720764160156253 Accuracy: 0.6666666666666666
